# Combining Data

In [2]:
# Imports
import pandas as pd

In [3]:
# Reading in data
irs = pd.read_csv('../data/irs_clean_2.csv')
city = pd.read_csv('../data/avg_price_restaurants.csv')
city_2 = pd.read_csv('../data/perc_price_category.csv')

## IRS Data

### Grouping IRS Data by City

In [9]:
irs.head()

,zipcode,income_bucket,n_returns,agi,total_income,city
0,6001,1,2100,19313,19920,Avon
1,6001,2,990,36486,37528,Avon
2,6001,3,910,56853,58191,Avon
3,6001,4,860,74414,75762,Avon
4,6001,5,2210,316572,324034,Avon


In [10]:
# Creating a map for the total of returns per city
total_returns_map = irs.groupby('city').sum()['n_returns']

In [11]:
# Creating a column from the map for the total number of returns per city
irs['total_n_returns'] = irs['city'].map(total_returns_map)

In [12]:
irs.head()

,zipcode,income_bucket,n_returns,agi,total_income,city,total_n_returns
0,6001,1,2100,19313,19920,Avon,9260
1,6001,2,990,36486,37528,Avon,9260
2,6001,3,910,56853,58191,Avon,9260
3,6001,4,860,74414,75762,Avon,9260
4,6001,5,2210,316572,324034,Avon,9260


In [13]:
# Calculating the weighted average of the income bucket per city weighted by the number of returns
irs['y'] = irs['n_returns'] / irs['total_n_returns'] * irs['income_bucket']

In [14]:
# Grouping by city to merge with restaurant data
y = pd.DataFrame(irs.groupby('city').sum()['y'].round(0))

In [15]:
y.head()

,y
city,
Amston,3.0
Andover,3.0
Ansonia,2.0
Ashford,3.0
Avon,4.0


In [16]:
# Resetting the index so that the index is not cities
y.reset_index(inplace=True)

### Merging IRS Data with Average Price Restaurant Data

In [22]:
city.head()

,city,price,num_restaurants
0,Amston,1.0,1
1,Andover,1.0,3
2,Ansonia,2.0,12
3,Ashford,1.5,4
4,Avon,2.1,30


In [23]:
city.shape

(227, 3)

In [24]:
y.shape

(216, 2)

**Note:** the shapes do not match up, so some cities in our city data are not represented in the irs data.

In [25]:
# Checking which cities are missing from the IRS data
missing_city = []
for c in city['city'].values:
    if c not in y['city'].values:
        missing_city.append(c)

missing_city

['Bethlehem',
 'Cobalt',
 'Collinsville',
 'Durham',
 'East WIndsor',
 'Georgetown',
 'Griswold',
 'Hadlyme',
 'Hartland',
 'Harwinton',
 'Ivoryton',
 'Kensington',
 'Killingly',
 'Lisbon',
 'Mansfield',
 'Mashantucket',
 'Milldale',
 'Montville',
 'Newent',
 'Noank',
 'Old Mystic',
 'Pomfret',
 'Rockville',
 'Rowayton',
 'Somersville',
 'South WIndsor',
 'Stafford',
 'Stevenson',
 'Storrs',
 'Union',
 'Vernon',
 'Winchester',
 'Yantic']

In [26]:
# Checking which cities are missing from the city data
missing_city_2 = []
for c in y['city'].values:
    if c not in city['city'].values:
        missing_city_2.append(c)

missing_city_2

['Bethlehem Village',
 'Cornwall Bridge',
 'East Canaan',
 'East Hartland',
 'East Killingly',
 'Hampton',
 'Ivorytown',
 'Mansfield Center',
 'North Granby',
 'Northfield',
 'Northford',
 'Northwest Harwinton',
 'Oxoboxo River',
 'Pawcatuck',
 'Pomfret Center',
 'Quinebaug',
 'Roxbury',
 'Scotland',
 'South Kent',
 'West Granby',
 'West Suffield',
 'Woodstock Valley']

**Note:** some cities, such as E. Hartford (East Hartford) may be typed out differently. There are also some that have random caps ("South WIndsor"). Other cities may just actually not be contained in both datasets. I will lower case all and clean up some of the city names to make sure we line up the cities that are in both datasets.

In [27]:
# Capitalizing the first letter of every word, which is how cities are represented in the irs data
city['city'] = city['city'].str.title()

In [28]:
# Dropping cities in the restaurant data that are not in the irs data
city = city.loc[city['city'].isin(y['city'])]

In [29]:
# Dropping cities in the irs data that are not in the restaurant data
y = y.loc[y['city'].isin(city['city'])]

In [30]:
city.shape

(196, 3)

In [31]:
y.shape

(194, 2)

**Note:** the restaurant data still contains two additional cities. These are likely duplicates.

In [32]:
# Dropping duplicates
city = city.drop_duplicates(subset = 'city')

In [33]:
city.shape

(194, 3)

In [36]:
# Merging both data frames on city
df_1 = city.merge(y, on='city')

In [37]:
# Taking a look at the data
df_1.head()

,city,price,num_restaurants,y
0,Amston,1.0,1,3.0
1,Andover,1.0,3,3.0
2,Ansonia,2.0,12,2.0
3,Ashford,1.5,4,3.0
4,Avon,2.1,30,4.0


#### Export

In [38]:
df_1.to_csv('../data/combined_1.csv', index=False)

### Merging IRS Data with Price Category Restaurant Data

In [39]:
city_2.head()

,city,1,2,3,4,total_restaurants
0,Amston,1.000000,0.000000,0.000000,0.0,1
1,Andover,1.000000,0.000000,0.000000,0.0,3
2,Ansonia,0.000000,1.000000,0.000000,0.0,12
3,Ashford,0.500000,0.500000,0.000000,0.0,4
4,Avon,0.033333,0.833333,0.133333,0.0,30


In [40]:
city_2.shape

(227, 6)

In [41]:
y.shape

(194, 2)

**Note:** the shapes do not match up, so we will go through the same process we did for the other data.

In [44]:
# Capitalizing the first letter of every word, which is how cities are represented in the irs data
city_2['city'] = city_2['city'].str.title()

In [45]:
# Dropping cities in the restaurant data that are not in the irs data
city_2 = city_2.loc[city_2['city'].isin(y['city'])]

In [46]:
# Dropping cities in the irs data that are not in the restaurant data
y = y.loc[y['city'].isin(city_2['city'])]

In [52]:
# Dropping duplicates
city_2 = city_2.drop_duplicates(subset = 'city')

In [53]:
city_2.shape

(194, 6)

In [54]:
y.shape

(194, 2)

In [55]:
# Merging both data frames on city
df_2 = city_2.merge(y, on='city')

In [56]:
# Taking a look at the data
df_2.head()

,city,1,2,3,4,total_restaurants,y
0,Amston,1.000000,0.000000,0.000000,0.0,1,3.0
1,Andover,1.000000,0.000000,0.000000,0.0,3,3.0
2,Ansonia,0.000000,1.000000,0.000000,0.0,12,2.0
3,Ashford,0.500000,0.500000,0.000000,0.0,4,3.0
4,Avon,0.033333,0.833333,0.133333,0.0,30,4.0


#### Export

In [57]:
df_2.to_csv('../data/combined_2.csv', index=False)

## CT Income Data

In [3]:
# Reading in the data
ct_df = pd.read_csv('../data/ct_income_clean.csv')
city = pd.read_csv('../data/avg_price_restaurants.csv')
city_2 = pd.read_csv('../data/perc_price_category.csv')

In [4]:
# Taking a look at the data
ct_df.head()

,city,median_household_income,mean_household_income,per_capita_income,median_bucket,mean_bucket
0,Andover,100321,111230,40182,5,5
1,Ansonia,43305,62858,24359,2,3
2,Ashford,77870,95339,39139,4,4
3,Avon,123894,172245,66822,5,5
4,Barkhamsted,95735,102210,40156,4,5


### Merging CT Data with Average Price Restaurant Data

In [5]:
city.head()

,city,price,num_restaurants
0,Amston,1.0,1
1,Andover,1.0,3
2,Ansonia,2.0,12
3,Ashford,1.5,4
4,Avon,2.1,30


In [6]:
city.shape

(227, 3)

In [7]:
ct_df.shape

(169, 6)

In [8]:
# Shapes do not match, so we will go through the same process as before
city = city.loc[city['city'].isin(ct_df['city'])]
ct_df = ct_df.loc[ct_df['city'].isin(city['city'])]

In [9]:
city.shape

(160, 3)

In [10]:
ct_df.shape

(160, 6)

In [11]:
# Merging the data frames on city
df_3 = city.merge(ct_df, on='city')

In [12]:
# Taking a look at the data
df_3.head()

,city,price,num_restaurants,median_household_income,mean_household_income,per_capita_income,median_bucket,mean_bucket
0,Andover,1.0,3,100321,111230,40182,5,5
1,Ansonia,2.0,12,43305,62858,24359,2,3
2,Ashford,1.5,4,77870,95339,39139,4,4
3,Avon,2.1,30,123894,172245,66822,5,5
4,Barkhamsted,1.0,2,95735,102210,40156,4,5


#### Export

In [13]:
df_3.to_csv('../data/combined_3.csv', index=False)

### Merging CT Data with Price Category Restaurant Data

In [14]:
city_2.head()

,city,1,2,3,4,total_restaurants
0,Amston,1.000000,0.000000,0.000000,0.0,1
1,Andover,1.000000,0.000000,0.000000,0.0,3
2,Ansonia,0.000000,1.000000,0.000000,0.0,12
3,Ashford,0.500000,0.500000,0.000000,0.0,4
4,Avon,0.033333,0.833333,0.133333,0.0,30


In [15]:
city_2.shape

(227, 6)

In [16]:
ct_df.shape

(160, 6)

In [17]:
# Shapes do not match, so we will go through the same process as before
city_2 = city_2.loc[city_2['city'].isin(ct_df['city'])]
ct_df = ct_df.loc[ct_df['city'].isin(city_2['city'])]

In [18]:
city_2.shape

(160, 6)

In [19]:
ct_df.shape

(160, 6)

In [20]:
# Merging the data frames on city
df_4 = city_2.merge(ct_df, on='city')

In [21]:
# Taking a look at the data
df_4.head()

,city,1,2,3,4,total_restaurants,median_household_income,mean_household_income,per_capita_income,median_bucket,mean_bucket
0,Andover,1.000000,0.000000,0.000000,0.0,3,100321,111230,40182,5,5
1,Ansonia,0.000000,1.000000,0.000000,0.0,12,43305,62858,24359,2,3
2,Ashford,0.500000,0.500000,0.000000,0.0,4,77870,95339,39139,4,4
3,Avon,0.033333,0.833333,0.133333,0.0,30,123894,172245,66822,5,5
4,Barkhamsted,1.000000,0.000000,0.000000,0.0,2,95735,102210,40156,4,5


#### Export

In [22]:
df_4.to_csv('../data/combined_4.csv', index=False)